In [3]:
# https://colab.research.google.com/drive/1bZeL8tReU2HmZUl0iIQJTDcAjc6M3M-k#gceVm=dbda-379505/us-west1-b/vmforcollab-vm

In [4]:
#!pip install -r /requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=108de5d46c1313fba94014288da18213e9dcdc87079d7bf57aa0b9b379af7db7
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Row

#import findspark
#findspark.init()
mongodb_spark_jars_path=r'/jars'
input_file_path=r'/vehicle_collision_data.csv'

In [6]:
# spark=SparkSession.builder.config("spark.mongodb.input.uri","mongodb://localhost:27017/test").config("spark.mongodb.output.uri","mongodb://localhost:27017/mydb.spotify").config("spark.mongodb.input.uri","mongodb://localhost:27017/test.employee").config("spark.mongodb.output.uri","mongodb://localhost:27017/test.employee").config("spark.jars", ""+mongodb_spark_jars_path+"\\mongo-spark-connector_2.12-3.0.1.jar,"+mongodb_spark_jars_path+"\\bson-4.9.0.jar,"+mongodb_spark_jars_path+"\\mongodb-driver-core-4.0.5.jar,"+mongodb_spark_jars_path+"\\mongodb-driver-sync-4.0.5.jar").getOrCreate()



# test3 working
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("merged").config("spark.jars", "../jars/").config("spark.driver.memory", "8g").config("spark.executor.memory", "8g").getOrCreate()

# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .appName("MyApp") \
#     .config("spark.driver.memory", "8g") \
#     .config("spark.executor.memory", "8g") \
#     .getOrCreate()


spark

In [7]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType, FloatType
from datetime import datetime
from pyspark.sql.types import * 
schema=StructType([     StructField("crash_date",StringType(),True), 
                        StructField("crash_time",StringType(),True),
                        StructField("borough",StringType(),True),
                        StructField("zip_code",IntegerType(),True), 
                        StructField("latitude",IntegerType(),True), 
                        StructField("longitude",IntegerType(),True), 
                        StructField("location",StringType(),True),
                        StructField("street_name",StringType(),True),
                        StructField("cross_street_name",StringType(),True),
                        StructField("off_street_name",StringType(),True),
                        StructField("number_of_people_injured",IntegerType(),True), 
                        StructField("number_of_people_killed",IntegerType(),True), 
                        StructField("number_of_pedestrians_injured",IntegerType(),True),
                        StructField("number_of_pedestrians_killed",IntegerType(),True), 
                        StructField("number_of_cyclist_injured",IntegerType(),True), 
                        StructField("number_of_cyclist_killed",IntegerType(),True), 
                        StructField("number_of_motorist_injured",IntegerType(),True),
                        StructField("number_of_motorist_killed",IntegerType(),True), 
                        StructField("contributing_factor_vehicle_1",StringType(),True), 
                        StructField("contributing_factor_vehicle_2",StringType(),True), 
                        StructField("contributing_factor_vehicle_3",StringType(),True),
                        StructField("contributing_factor_vehicle_4",StringType(),True),
                        StructField("contributing_factor_vehicle_5",StringType(),True),
                        StructField("collision_id",IntegerType(),True),
                        StructField("vehicle_type_code_1",StringType(),True), 
                        StructField("vehicle_type_code_2",StringType(),True), 
                        StructField("vehicle_type_code_3",StringType(),True),
                        StructField("vehicle_type_code_4",StringType(),True), 
                        StructField("vehicle_type_code_5",StringType(),True) 
                
                    ])

In [8]:
crash = spark.read.option("header", "true").schema(schema).csv(input_file_path)

In [9]:
crash.printSchema()

root
 |-- crash_date: string (nullable = true)
 |-- crash_time: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- latitude: integer (nullable = true)
 |-- longitude: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- cross_street_name: string (nullable = true)
 |-- off_street_name: string (nullable = true)
 |-- number_of_people_injured: integer (nullable = true)
 |-- number_of_people_killed: integer (nullable = true)
 |-- number_of_pedestrians_injured: integer (nullable = true)
 |-- number_of_pedestrians_killed: integer (nullable = true)
 |-- number_of_cyclist_injured: integer (nullable = true)
 |-- number_of_cyclist_killed: integer (nullable = true)
 |-- number_of_motorist_injured: integer (nullable = true)
 |-- number_of_motorist_killed: integer (nullable = true)
 |-- contributing_factor_vehicle_1: string (nullable = true)
 |-- contributing_factor_vehicle_2: stri

In [10]:
from pyspark.sql.functions import *
from pyspark.sql.functions import concat, lit, col
crash = crash.withColumn('hour', split(crash['crash_time'], ':').getItem(0)).select( "*",concat(col("crash_date"),lit(" "),col("hour"),lit(":00")).alias("crash_date_time"))
crash.show(truncate=False)

+----------+----------+---------+--------+--------+---------+-----------------------+----------------------------+------------------------------+---------------------------+------------------------+-----------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+------------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-----------------------------------+-----------------------------------+-------------------+-------------------+-------------------+----+----------------+
|crash_date|crash_time|borough  |zip_code|latitude|longitude|location               |street_name                 |cross_street_name             |off_street_name            |number_of_people_injured|number_of_people_killed|number_of_pedestrians_injured|number_of_pedestrians_killed|number_of_cycli

In [11]:
crash.printSchema()

root
 |-- crash_date: string (nullable = true)
 |-- crash_time: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- latitude: integer (nullable = true)
 |-- longitude: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- cross_street_name: string (nullable = true)
 |-- off_street_name: string (nullable = true)
 |-- number_of_people_injured: integer (nullable = true)
 |-- number_of_people_killed: integer (nullable = true)
 |-- number_of_pedestrians_injured: integer (nullable = true)
 |-- number_of_pedestrians_killed: integer (nullable = true)
 |-- number_of_cyclist_injured: integer (nullable = true)
 |-- number_of_cyclist_killed: integer (nullable = true)
 |-- number_of_motorist_injured: integer (nullable = true)
 |-- number_of_motorist_killed: integer (nullable = true)
 |-- contributing_factor_vehicle_1: string (nullable = true)
 |-- contributing_factor_vehicle_2: stri

In [12]:
crash = crash.drop('hour')

In [13]:
crash.printSchema()

root
 |-- crash_date: string (nullable = true)
 |-- crash_time: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- latitude: integer (nullable = true)
 |-- longitude: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- cross_street_name: string (nullable = true)
 |-- off_street_name: string (nullable = true)
 |-- number_of_people_injured: integer (nullable = true)
 |-- number_of_people_killed: integer (nullable = true)
 |-- number_of_pedestrians_injured: integer (nullable = true)
 |-- number_of_pedestrians_killed: integer (nullable = true)
 |-- number_of_cyclist_injured: integer (nullable = true)
 |-- number_of_cyclist_killed: integer (nullable = true)
 |-- number_of_motorist_injured: integer (nullable = true)
 |-- number_of_motorist_killed: integer (nullable = true)
 |-- contributing_factor_vehicle_1: string (nullable = true)
 |-- contributing_factor_vehicle_2: stri

In [14]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Row

In [15]:
#loading weather data

weather_data =spark.read.csv(r'/weather_scraping.csv',inferSchema=True, header=True).toDF('date','timestamp','time_of_day','temperature','condition','pressure','dew_point','humidity','visibility_in_miles',
                                                                                          'wind_chill_temp','wdir_cardinal','wind_gust','wind_speed','maximum_temp','minimum_temp','precipitation_rate','snow_rate')

In [16]:
weather_data = weather_data.where(weather_data.condition!='0')

In [17]:
weather_data.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- time_of_day: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- condition: string (nullable = true)
 |-- pressure: string (nullable = true)
 |-- dew_point: integer (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- visibility_in_miles: double (nullable = true)
 |-- wind_chill_temp: integer (nullable = true)
 |-- wdir_cardinal: string (nullable = true)
 |-- wind_gust: integer (nullable = true)
 |-- wind_speed: integer (nullable = true)
 |-- maximum_temp: integer (nullable = true)
 |-- minimum_temp: integer (nullable = true)
 |-- precipitation_rate: double (nullable = true)
 |-- snow_rate: integer (nullable = true)



In [18]:
from pyspark.sql.functions import concat, lit, col
weather_data = weather_data.withColumn("hour", F.hour(F.col("timestamp")))

weather_data.show(1)
# we get hour column here

+-------------------+-------------------+-----------+-----------+-------------+--------+---------+--------+-------------------+---------------+-------------+---------+----------+------------+------------+------------------+---------+----+
|               date|          timestamp|time_of_day|temperature|    condition|pressure|dew_point|humidity|visibility_in_miles|wind_chill_temp|wdir_cardinal|wind_gust|wind_speed|maximum_temp|minimum_temp|precipitation_rate|snow_rate|hour|
+-------------------+-------------------+-----------+-----------+-------------+--------+---------+--------+-------------------+---------------+-------------+---------+----------+------------+------------+------------------+---------+----+
|2016-03-16 00:00:00|2023-03-03 10:21:00|          N|         50|Mostly Cloudy|    null|       46|      86|               10.0|             50|            E|     null|         3|          57|          45|               0.0|     null|  10|
+-------------------+-------------------+---

In [19]:
# we format date column because it has 00:00:00 as timestamp value, which is not usefull for us
# also format date as mm/dd/yyyy to match with crash dataset
weather = weather_data.withColumn("date", F.date_format("date", "MM/dd/yyyy"))
weather.show(1)

+----------+-------------------+-----------+-----------+-------------+--------+---------+--------+-------------------+---------------+-------------+---------+----------+------------+------------+------------------+---------+----+
|      date|          timestamp|time_of_day|temperature|    condition|pressure|dew_point|humidity|visibility_in_miles|wind_chill_temp|wdir_cardinal|wind_gust|wind_speed|maximum_temp|minimum_temp|precipitation_rate|snow_rate|hour|
+----------+-------------------+-----------+-----------+-------------+--------+---------+--------+-------------------+---------------+-------------+---------+----------+------------+------------+------------------+---------+----+
|03/16/2016|2023-03-03 10:21:00|          N|         50|Mostly Cloudy|    null|       46|      86|               10.0|             50|            E|     null|         3|          57|          45|               0.0|     null|  10|
+----------+-------------------+-----------+-----------+-------------+--------+-

In [20]:
#weather_details = weather.select( "*",concat(col("date"),lit(" "),col("hour"),lit(":00")).alias("weather_date_time"))
weather_details = weather.select( "*",concat(col("date"),lit(" "),col("hour"),lit(":00")).alias("weather_date_time"))
weather_details.show(3)

+----------+-------------------+-----------+-----------+-------------+---------------+---------+--------+-------------------+---------------+-------------+---------+----------+------------+------------+------------------+---------+----+-----------------+
|      date|          timestamp|time_of_day|temperature|    condition|       pressure|dew_point|humidity|visibility_in_miles|wind_chill_temp|wdir_cardinal|wind_gust|wind_speed|maximum_temp|minimum_temp|precipitation_rate|snow_rate|hour|weather_date_time|
+----------+-------------------+-----------+-----------+-------------+---------------+---------+--------+-------------------+---------------+-------------+---------+----------+------------+------------+------------------+---------+----+-----------------+
|03/16/2016|2023-03-03 10:21:00|          N|         50|Mostly Cloudy|           null|       46|      86|               10.0|             50|            E|     null|         3|          57|          45|               0.0|     null|  10

In [21]:
weather_details.show()

+----------+-------------------+-----------+-----------+--------------------+---------------+---------+--------+-------------------+---------------+-------------+---------+----------+------------+------------+------------------+---------+----+-----------------+
|      date|          timestamp|time_of_day|temperature|           condition|       pressure|dew_point|humidity|visibility_in_miles|wind_chill_temp|wdir_cardinal|wind_gust|wind_speed|maximum_temp|minimum_temp|precipitation_rate|snow_rate|hour|weather_date_time|
+----------+-------------------+-----------+-----------+--------------------+---------------+---------+--------+-------------------+---------------+-------------+---------+----------+------------+------------+------------------+---------+----+-----------------+
|03/16/2016|2023-03-03 10:21:00|          N|         50|       Mostly Cloudy|           null|       46|      86|               10.0|             50|            E|     null|         3|          57|          45|     

## **merge datasets on common date_time column**

In [22]:
merged_data = crash.join(weather_details, crash["crash_date_time"] == weather_details["weather_date_time"],'inner')

In [23]:
merged_data.columns

['crash_date',
 'crash_time',
 'borough',
 'zip_code',
 'latitude',
 'longitude',
 'location',
 'street_name',
 'cross_street_name',
 'off_street_name',
 'number_of_people_injured',
 'number_of_people_killed',
 'number_of_pedestrians_injured',
 'number_of_pedestrians_killed',
 'number_of_cyclist_injured',
 'number_of_cyclist_killed',
 'number_of_motorist_injured',
 'number_of_motorist_killed',
 'contributing_factor_vehicle_1',
 'contributing_factor_vehicle_2',
 'contributing_factor_vehicle_3',
 'contributing_factor_vehicle_4',
 'contributing_factor_vehicle_5',
 'collision_id',
 'vehicle_type_code_1',
 'vehicle_type_code_2',
 'vehicle_type_code_3',
 'vehicle_type_code_4',
 'vehicle_type_code_5',
 'crash_date_time',
 'date',
 'timestamp',
 'time_of_day',
 'temperature',
 'condition',
 'pressure',
 'dew_point',
 'humidity',
 'visibility_in_miles',
 'wind_chill_temp',
 'wdir_cardinal',
 'wind_gust',
 'wind_speed',
 'maximum_temp',
 'minimum_temp',
 'precipitation_rate',
 'snow_rate',
 'hou

In [24]:
merged_data.printSchema()

root
 |-- crash_date: string (nullable = true)
 |-- crash_time: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- latitude: integer (nullable = true)
 |-- longitude: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- cross_street_name: string (nullable = true)
 |-- off_street_name: string (nullable = true)
 |-- number_of_people_injured: integer (nullable = true)
 |-- number_of_people_killed: integer (nullable = true)
 |-- number_of_pedestrians_injured: integer (nullable = true)
 |-- number_of_pedestrians_killed: integer (nullable = true)
 |-- number_of_cyclist_injured: integer (nullable = true)
 |-- number_of_cyclist_killed: integer (nullable = true)
 |-- number_of_motorist_injured: integer (nullable = true)
 |-- number_of_motorist_killed: integer (nullable = true)
 |-- contributing_factor_vehicle_1: string (nullable = true)
 |-- contributing_factor_vehicle_2: stri

In [25]:
merged_data.columns

['crash_date',
 'crash_time',
 'borough',
 'zip_code',
 'latitude',
 'longitude',
 'location',
 'street_name',
 'cross_street_name',
 'off_street_name',
 'number_of_people_injured',
 'number_of_people_killed',
 'number_of_pedestrians_injured',
 'number_of_pedestrians_killed',
 'number_of_cyclist_injured',
 'number_of_cyclist_killed',
 'number_of_motorist_injured',
 'number_of_motorist_killed',
 'contributing_factor_vehicle_1',
 'contributing_factor_vehicle_2',
 'contributing_factor_vehicle_3',
 'contributing_factor_vehicle_4',
 'contributing_factor_vehicle_5',
 'collision_id',
 'vehicle_type_code_1',
 'vehicle_type_code_2',
 'vehicle_type_code_3',
 'vehicle_type_code_4',
 'vehicle_type_code_5',
 'crash_date_time',
 'date',
 'timestamp',
 'time_of_day',
 'temperature',
 'condition',
 'pressure',
 'dew_point',
 'humidity',
 'visibility_in_miles',
 'wind_chill_temp',
 'wdir_cardinal',
 'wind_gust',
 'wind_speed',
 'maximum_temp',
 'minimum_temp',
 'precipitation_rate',
 'snow_rate',
 'hou

In [37]:
merged_data.show()

+----------+----------+---------+--------+--------+---------+--------------------+--------------------+--------------------+--------------------+------------------------+-----------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+----------------+----------+-------------------+-----------+-----------+-------------+---------------+---------+--------+-------------------+---------------+-------------+---------+----------+------------+------------+------------------+---------+----+-----------------+
|crash_date|crash_time|  borough|zip_code|latitude|longitude|            location|         street_name|   c

In [27]:
# !pip install pyspark py4j

In [28]:
# final_data = merged_data.toPandas()


In [29]:
#final_data

## **push data to mongodb**

In [30]:
#!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import pymongo
# from pyspark.sql import SparkSession
# from pymongo import MongoClient
# from pyspark.sql.functions import to_json
# import json


# client  = pymongo.MongoClient('<your_mongodb_cluster_id>')

    
# db = client['crash_weather']
# collection = db['merged_data']


# data = [row.asDict() for row in merged_data.collect()] # for pyspark df to dict

# collection.insert_many(data)

In [34]:
merged_data.count()

85311

In [42]:
import sys
sys.getsizeof(data)

732808